In [2]:
import numpy as np

In [3]:
def initialize_kernel(size):
    val = []
    val.extend([1]*(size//2))
    val.extend([0])
    val.extend([-1]*(size//2))
    return [val]*size

In [4]:
def pad(x,pad_size):
    size = len(x)
    val = [0]*(size+(2*pad_size))
    arr = []
    for _ in range(pad_size):
        arr.append(val)
    for i in x:
        a = []
        a.extend([0]*pad_size)
        a.extend(i)
        a.extend([0]*pad_size)
        arr.append(a)
    for _ in range(pad_size):
        arr.append(val)
    print(f'\nAfter padding size = ({len(arr)},{len(arr[0])})')
    return arr


In [5]:
def conv(x,y):
    val = 0
    for i,j in zip(x,y):
        for val_i,val_j in zip(i,j):
            val+= val_i*val_j
    return val

In [6]:
def column_sep(row_values,start,end):
    arr = []
    for i in row_values:
        arr.append(i[start:end])
    return arr

In [7]:
def conv_layer(x,p,s,kernel_size):
    x = pad(x,p)
    print('After padding:')
    print(x)
    kernel = initialize_kernel(kernel_size)
    print('\nKernel Initialization Done')
    print(kernel)
    output_size = (len(x)-kernel_size)//s + 1
    out = [[0]*output_size]*output_size
    start_w = 0
    final_result = []
    for i in range(output_size):
        arr = []
        start_h = 0
        if i!=0:
            start_w+=s
        row_values = x[start_w:start_w+kernel_size]
        for j in range(output_size):
            if j!=0:
                start_h+=s
            col_values = column_sep(row_values,start_h,start_h+kernel_size)
            arr.append(conv(col_values,kernel))
        final_result.append(arr)
    return final_result

In [8]:
def pool(X,type='max'):
    if type=='max':
        maxi = X[0][0]
        for i in X:
            for j in i:
                if j>maxi:
                    maxi=j
        return maxi
    elif type=='avg':
        val = 0
        ctr = 0
        for i in X:
            for j in i:
                val+=j
                ctr+=1
        return val/ctr

In [9]:
def pool_layer(x,s,kernel_size,type):
    output_size = (len(x)-kernel_size)//s + 1
    out = [[0]*output_size]*output_size
    start_w = 0
    final_result = []
    for i in range(output_size):
        arr = []
        start_h = 0
        if i!=0:
            start_w+=s
        row_values = x[start_w:start_w+kernel_size]
        for j in range(output_size):
            if j!=0:
                start_h+=s
            col_values = column_sep(row_values,start_h,start_h+kernel_size)
            arr.append(pool(col_values,type=type))
        final_result.append(arr)
    return final_result

In [10]:
def Flatten(X):
    arr = []
    for x in X:
        for val in x:
            arr.append(val)
    return arr

In [11]:
import random
def weight_initialize(input_size,nodes):
    w = []
    for x in range(nodes):
        arr = []
        for y in range(input_size):
            arr.append(random.random())
        w.append(arr)
    return w

In [12]:
def bias_initialize(nodes):
    b = []
    for x in range(nodes):
        b.append(random.random())
    return b

In [13]:
def sigmoid(val):
    return 1/(1+np.exp(val*-1))

In [14]:
def fc(values,nodes):
    w = weight_initialize(len(values),nodes)
    print('\nWeight Initialization Done')
    print(w)
    b = bias_initialize(nodes)
    print('\nBias Initialization Done')
    print(b)
    val = np.dot(np.array(values),np.array(w).T)+b
    return val

In [15]:
def fc_dummy(values,nodes):
    w = [[1]*len(values)]*nodes
    b = [1]*nodes
    val = np.dot(np.array(values),np.array(w).T)+b
    return val

In [16]:
X = [[1,2,3,4,5],
     [6,7,8,9,10],
     [11,12,13,14,15],
     [16,17,18,19,20],
     [21,22,23,24,25]]

print(f'Orignal Data Shape : {len(X),len(X[0])}')
print('Input data :')
print(X)


c1 = conv_layer(X,p=0,s=1,kernel_size=3)
print('\nAfter Convolution')
print(c1)

p1 = pool_layer(c1,s=1,kernel_size=2,type='max')
print('\nAfter Pooling')
print(p1)

f1 = Flatten(p1)
print('\nAfter Flatten')
print(f1)

fc1 = fc(f1,2)
print('\nAfter Hidden Layer Output')
print(fc1)

o1 = fc(fc1,1)
print('\nOutput at Output layer before sigmoid function')
print(o1)
print('\nOutput at Output layer after sigmoid function')
print(sigmoid(o1))

Orignal Data Shape : (5, 5)
Input data :
[[1, 2, 3, 4, 5], [6, 7, 8, 9, 10], [11, 12, 13, 14, 15], [16, 17, 18, 19, 20], [21, 22, 23, 24, 25]]

After padding size = (5,5)
After padding:
[[1, 2, 3, 4, 5], [6, 7, 8, 9, 10], [11, 12, 13, 14, 15], [16, 17, 18, 19, 20], [21, 22, 23, 24, 25]]

Kernel Initialization Done
[[1, 0, -1], [1, 0, -1], [1, 0, -1]]

After Convolution
[[-6, -6, -6], [-6, -6, -6], [-6, -6, -6]]

After Pooling
[[-6, -6], [-6, -6]]

After Flatten
[-6, -6, -6, -6]

Weight Initialization Done
[[0.7517166499455614, 0.18049203224014632, 0.5811065533778713, 0.13441733512599208], [0.8221667745309269, 0.6386915825386187, 0.6421523807028915, 0.21018336725914377]]

Bias Initialization Done
[0.6636059540574833, 0.22088228108197805]

After Hidden Layer Output
[ -9.22278947 -13.65828235]

Weight Initialization Done
[[0.1593419873293126, 0.8043455875020854]]

Bias Initialization Done
[0.9112261575823334]

Output at Output layer before sigmoid function
[-11.54433059]

Output at Output

In [17]:
X = [[1,2,3,4,5],
     [6,7,8,9,10],
     [11,12,13,14,15],
     [16,17,18,19,20],
     [21,22,23,24,25]]

print(f'Orignal Data Shape : {len(X),len(X[0])}')
print('Input data :')
print(X)


c1 = conv_layer(X,p=0,s=1,kernel_size=3)
print('\nAfter Convolution')
print(c1)

p1 = pool_layer(c1,s=1,kernel_size=2,type='max')
print('\nAfter Pooling')
print(p1)

f1 = Flatten(p1)
print('\nAfter Flatten')
print(f1)

fc1 = fc_dummy(f1,2)
print('\nAfter Hidden Layer Output')
print(fc1)

o1 = fc_dummy(fc1,1)
print('\nOutput at Output layer before sigmoid function')
print(o1)
print('\nOutput at Output layer after sigmoid function')
print(sigmoid(o1))

Orignal Data Shape : (5, 5)
Input data :
[[1, 2, 3, 4, 5], [6, 7, 8, 9, 10], [11, 12, 13, 14, 15], [16, 17, 18, 19, 20], [21, 22, 23, 24, 25]]

After padding size = (5,5)
After padding:
[[1, 2, 3, 4, 5], [6, 7, 8, 9, 10], [11, 12, 13, 14, 15], [16, 17, 18, 19, 20], [21, 22, 23, 24, 25]]

Kernel Initialization Done
[[1, 0, -1], [1, 0, -1], [1, 0, -1]]

After Convolution
[[-6, -6, -6], [-6, -6, -6], [-6, -6, -6]]

After Pooling
[[-6, -6], [-6, -6]]

After Flatten
[-6, -6, -6, -6]

After Hidden Layer Output
[-23 -23]

Output at Output layer before sigmoid function
[-45]

Output at Output layer after sigmoid function
[2.86251858e-20]
